In [1]:
import datetime
import os
import shutil

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.metrics import MeanAbsolutePercentageError, MeanAbsoluteError, RootMeanSquaredError

import definitions
from training import train, data
from training.loguniform import LogUniform
from training.stepuniform import StepUniform
from training.steploguniform import StepLogUniform
from scipy.stats.distributions import randint
import numpy as np
import pandas as pd

import altair as alt

#alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
dataset = 'H125'
target = 'nu'
pad_target = 'W'
model_version = 'v2'
mix_weights = [0.0, 1.0, 10.0]

In [3]:
jigsaw_train, jigsaw_val, jigsaw_test = data.get_jigsaw(dataset=dataset, target=target)
x_train, y_train, x_val, y_val, x_test, y_test = data.get_datasets(dataset=dataset, target=target, scale_x=True, pad_target=pad_target)
print("x_train:")
print(x_train)
print(f"num training samples: {x_train.shape[0]}")
print(f"num validation samples: {x_val.shape[0]}")
print(f"num testing samples: {x_test.shape[0]}")

x_mean = [-0.21931746  0.01414732  0.07634757  0.03271993  0.17313513  0.106
  0.1429699  -0.04686724  0.46652866  0.106     ]
x_std = [2.60189267e+01 2.58930368e+01 2.07010843e+01 2.07878627e+01
 8.37326392e+01 6.80289158e-14 2.07625359e+01 2.06946765e+01
 8.35104710e+01 6.80289158e-14]
x_train:
           METx      METy  Lax_reco  Lay_reco  Laz_reco  Lam_reco  Lbx_reco  \
0     -0.385684 -0.470236  1.491813  0.274558 -1.375876       1.0 -1.004071   
1      1.467444  0.562137 -2.107043 -1.113026 -2.014977       1.0  0.261854   
2      0.801318 -1.543514  0.433313  0.635599 -0.427432       1.0 -1.436216   
3     -0.778773 -0.233567  0.608453 -0.602136  0.962574       1.0  0.369280   
4      1.148450 -1.135467 -1.051204 -0.084590  0.421404       1.0 -0.391107   
...         ...       ...       ...       ...       ...       ...       ...   
79995 -1.331289 -0.571023  0.968558 -0.051121  0.364525       1.0  0.702637   
79996 -1.389753 -0.232370  0.482799 -0.414591  0.180601       1.0  1.2

In [4]:
Nax_gen_chart = alt.Chart(y_test).mark_bar().encode(alt.X(f"{definitions.TARGETS[dataset][target][0]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Nay_gen_chart = alt.Chart(y_test).mark_bar().encode(alt.X(f"{definitions.TARGETS[dataset][target][1]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Naz_gen_chart = alt.Chart(y_test).mark_bar().encode(alt.X(f"{definitions.TARGETS[dataset][target][2]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Nbz_gen_chart = alt.Chart(y_test).mark_bar().encode(alt.X(f"{definitions.TARGETS[dataset][target][3]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Nax_gen_chart | Nay_gen_chart | Naz_gen_chart | Nbz_gen_chart

alt.HConcatChart(...)

In [5]:
Nax_reco_chart = alt.Chart(jigsaw_test).mark_bar().encode(alt.X(f"{definitions.JIGSAW_TARGETS[dataset][target][0]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Nay_reco_chart = alt.Chart(jigsaw_test).mark_bar().encode(alt.X(f"{definitions.JIGSAW_TARGETS[dataset][target][1]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Naz_reco_chart = alt.Chart(jigsaw_test).mark_bar().encode(alt.X(f"{definitions.JIGSAW_TARGETS[dataset][target][2]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Nbz_reco_chart = alt.Chart(jigsaw_test).mark_bar().encode(alt.X(f"{definitions.JIGSAW_TARGETS[dataset][target][3]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")
Nax_reco_chart | Nay_reco_chart | Naz_reco_chart | Nbz_reco_chart

alt.HConcatChart(...)

## Results

In [6]:
df_train, df_val, df_test = data.get_datasets(dataset=dataset, target=target, pad_target=pad_target, x_y_split=False)

print(df_train)

Hx_gen  Hy_gen     Hz_gen  Hm_gen       Hx_reco       Hy_reco  \
0        -0.0     0.0 -462.33320   125.0  0.000000e+00  4.440892e-15   
1         0.0     0.0 -540.76140   125.0 -1.421085e-14  0.000000e+00   
2         0.0     0.0 -302.00290   125.0  0.000000e+00 -8.881784e-15   
3         0.0    -0.0  339.42230   125.0  8.881784e-15 -1.065814e-14   
4        -0.0     0.0  244.77860   125.0  1.776357e-14  4.440892e-16   
...       ...     ...        ...     ...           ...           ...   
79995     0.0     0.0  167.81920   125.0 -3.552714e-15 -1.776357e-15   
79996    -0.0    -0.0  -34.02435   125.0 -1.776357e-15  3.552714e-15   
79997    -0.0    -0.0  767.80310   125.0  0.000000e+00  0.000000e+00   
79998    -0.0    -0.0  -98.31551   125.0 -3.552714e-15  0.000000e+00   
79999     0.0     0.0  837.30960   125.0  0.000000e+00  3.552714e-15   

         Hz_reco    Hm_reco    Wax_gen    Way_gen  ...    Nbx_gen    Nby_gen  \
0     -263.27580  128.90760  33.930650  10.610390  ... -13.226

In [7]:
log_dir = train.get_mixed_log_dir(dataset, target, pad_target, model_version, mix_weight=0.0)
print(log_dir)
model = tf.keras.models.load_model(str(log_dir / 'best_model'), compile=False)

C:\Users\alexj\work\mass_regression\logs\H125\nu-W-mixed_v2-w0.0


ValueError: Inconsistent values for attr 'Tidx' DT_FLOAT vs. DT_INT32 while building NodeDef 'tf_op_layer_weighted_loss_1/Sum_1/weighted_loss_1/Sum_1' using Op<name=Sum; signature=input:T, reduction_indices:Tidx -> output:T; attr=keep_dims:bool,default=false; attr=T:type,allowed=[DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]; attr=Tidx:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]>